<a href="https://colab.research.google.com/github/SuperVerkaufer/Recommender/blob/main/Recommender_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# To create a simple interactive dashboard for the movie recommender system, built using Dash

In [2]:
# Import libraries
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
!pip install surprise
import surprise
from surprise import Reader, Dataset, SVD #, evaluate

import warnings; warnings.simplefilter('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505170 sha256=72baedb5e997684a9c864757d3ccef741a48f160a630aac8428e4b09afaf15df
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [5]:
#UPLOAD 3 DATASETS DOWNLOADED FROM KAGGLE STORED ON desktop (RECOMMENDER FOLDER) to Google Colab
reader = Reader()
from google.colab import files
uploaded = files.upload()

#SURPLUS CODE - DOWNLOAD data to dataframe
#ratings = pd.read_csv('../input/tmdb-ratings_small/tmdb_5000_movies.csv')
#rd = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
#rd = pd.read_csv('C:\Users\zoolanderng@yahoo.com.sg\Desktop\ratings_small.csv')
#rd = pd.read_csv('\Users\zoolanderng@yahoo.com.sg\Desktop\ratings_small.csv')
#rd = pd.read_csv(r'C:\\Users\\zoolanderng@yahoo.com.sg\\Desktop\\ratings_small.csv')
#rd = pd.read_csv(r'C:\Users\zoola\OneDrive\Desktop\Recommendation dashboard\ratings_small.csv')
#rd = pd.read_csv("C:\Users\zoola\Desktop\Recommender\ratings_small.csv")
#rd = pd.read_csv(r'C:\Users\zoolanderng@yahoo.com.sg\Desktop\ratings_small.csv')
#md= pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
#ld= pd.read_csv('/kaggle/input/the-movies-dataset/links.csv')
#ratings.head()
#rd.head(3)

Saving links.csv to links.csv


In [6]:
#Read in the dataframe
rd = pd.read_csv('ratings_small.csv')
md= pd.read_csv('movies_metadata.csv')
ld= pd.read_csv('links.csv')

In [7]:
#Install the required libraries:
!pip install dash
!pip install dash-core-components
!pip install dash-html-components
!pip install dash-table

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [8]:
#Import KNN and Train Test Split
from surprise import KNNWithMeans
from surprise.model_selection import train_test_split
from surprise import accuracy

In [9]:
# Prepare the data for Surprise library
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(rd[['userId', 'movieId', 'rating']], reader)

# Split the dataset into training and testing sets
trainset, testset = train_test_split(data, test_size=.25)

In [10]:
# Use KNNWithMeans as the algorithm
algo = KNNWithMeans(k=20, sim_options={'name': 'pearson_baseline', 'user_based': True})

# Train the algorithm on the data
#trainset = data.build_full_trainset()
#trainset = data
algo.fit(trainset)

# Make predictions on the test set
predictions = algo.test(testset)

accuracy.rmse(predictions)  # Compute RMSE
accuracy.mae(predictions)   # Compute MAE
accuracy.mse(predictions)   # Compute MSE

ENTIREtrain = data.build_full_trainset()

algo.fit(ENTIREtrain)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9362
MAE:  0.7105
MSE: 0.8765
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [11]:
# merge movie titles with rd, i.e. ratings & user file

#MERGE ratings, metadata, linking files into movie ratings joined
rd_ld = pd.merge(rd, ld, on='movieId')
#rd_ld.head(5)
#rd_ld.info()
rd_ld = rd_ld.rename(columns={'tmdbId': 'id'})
md['id'] = pd.to_numeric(md['id'], errors='coerce')
#print(md['id'].dtype)  # Output: float64
#md.info()
rd_ld_md = pd.merge(rd_ld, md, on='id')
movie_ratings_joined = rd_ld_md[['userId', 'movieId','id', 'title', 'budget','genres', 'popularity','revenue', 'production_countries', 'vote_average', 'rating']]

#CONVERT some fields to numeric from the merged dataset
movie_ratings_joined['popularity'] = pd.to_numeric(movie_ratings_joined['popularity'], errors='coerce')
print(movie_ratings_joined['popularity'].dtype)  # Output: float64
#movie_ratings_joined.info()

movie_ratings_joined['vote_average'] = pd.to_numeric(movie_ratings_joined['vote_average'], errors='coerce')
print(movie_ratings_joined['vote_average'].dtype)  # Output: float64
#movie_ratings_joined.info()

#top_recommendations = pd.DataFrame(recommendations, columns=['movieId', 'Rating'])
#top_recommendations.head(20)

#MERGE top recommendations with movie ratings joined
#mergedRD = pd.merge(top_recommendations[['movieId']], movie_ratings_joined[['movieId','title']], on='movieId', how='left')
#df_unique = mergedRD.drop_duplicates()
#df_unique

float64
float64


In [12]:
# Function to get recommendations for a user
def get_recommendations(user_id, num_recommendations=5):
  recommendations = []
  for movie_id in range(1, rd['movieId'].max() + 1):  # Iterate through all movie IDs
    if (user_id, movie_id) not in ENTIREtrain.ur[user_id]:  # Predict only for movies not rated by the user
    #if not rd[(rd['userId'] == user_id) & (rd['movieId'] == movie_id)].empty:
      prediction = algo.predict(user_id, movie_id)
      recommendations.append((movie_id, prediction.est))

  recommendations.sort(key=lambda x: x[1], reverse=True) # Sort by predicted rating

  top_recommendations = pd.DataFrame(recommendations, columns=['movieId', 'Rating'])
#top_recommendations.head(20)

#MERGE top recommendations with movie ratings joined
  mergedRD = pd.merge(top_recommendations[['movieId']], movie_ratings_joined[['movieId','title']], on='movieId', how='left')
  recommendations = mergedRD.drop_duplicates()
  recommendations

  return recommendations[:num_recommendations]
  #return tuple(recommendations[:num_recommendations].itertuples(index=False))

In [13]:
# Example usage: get recommendations for user with ID 1
recommendations = get_recommendations(1000)
recommendations

,movieId,title
0,1,Toy Story
247,2,Jumanji
354,3,Grumpier Old Men
413,4,Waiting to Exhale
426,5,Father of the Bride Part II


In [14]:
# Build Interactive Dashboard
# Import Dash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output

# Assuming 'get_recommendations', 'rd', 'trainset', 'algo', and 'movie_ratings_joined' are defined

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Movie Recommender Dashboard"),
    html.Label("Enter User ID:"),
    dcc.Input(id='user-id-input', type='number', value=1),  # Default user ID is 1
    html.Button('Get Recommendations', id='submit-button'),
    html.Div(id='recommendations-output')
])

@app.callback(
    Output('recommendations-output', 'children'),
    [Input('submit-button', 'n_clicks')],
    [dash.dependencies.State('user-id-input', 'value')]
)
def update_output(n_clicks, user_id):
    if n_clicks is not None:  # Check if the button is clicked
        try:
            user_id = int(user_id)  # Ensure user_id is an integer
            recommendations = get_recommendations(user_id)  # Get recommendations
            if recommendations.empty:
                return "No recommendations found for this user."
            else:
                return html.Ul([html.Li(title) for title in recommendations['title']])
        except ValueError:
            return "Invalid user ID. Please enter a number."
        except Exception as e:
            return f"An error occurred: {str(e)}"
    return ""  # Return empty string initially

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>